Here we just demonstrate how to declare an extension type with cdef class and how to define methods visible to the Python space (and still fast when being called in the C-scope)

In [1]:
%load_ext cython

We define the class Point, with the attributes x and y, declared as Float. cdef classes are faster than Python classes because data is arranged as C-structures instead of dictionaries (The lookup process is faster).

We declare both attributes as "public" for the Python space, so we can access them as regular Python attributes.

Next, length_squared and length methods are implemented. The first one is only avaliable at C-scope and the second is declared with the keyword "cpdef". That means, cython will implement 2 versions: One used when called by C-code and other one which can be invoked by Python (which is slower).

Finally, metamethods <span>__add__</span>, <span>__str__</span> and <span>__repr__</span> are implemented as regular "def" python methods.

In [66]:
%%cython --annotate

cimport cython
from libc.math cimport sqrt

cdef class Point:
    cdef public float x, y
    
    def __init__(self):
        self.x, self.y = 0.0, 0.0
    
    def __init__(self, float x, float y):
        self.x, self.y = x, y
    
    cdef float length_squared(self):
        return self.x ** 2 + self.y ** 2
    
    cpdef float length(self):
        return sqrt(self.length_squared())
    
    def __add__(self, Point other):
        return Point(self.x + other.x, self.y + other.y)
    
    # other can be 
    @cython.nonecheck(True)
    cpdef Point add(self, Point other):
        return Point(self.x + other.x, self.y + other.y)
    
    def __str__(self):
        return '(' + str(self.x) + ', ' + str(self.y) + ')'
    
    def __repr__(self):
        return str(self)

In [67]:
Point(1, 2).add(None)

AttributeError: 'NoneType' object has no attribute 'x'